# Ce jupyter s'appuie sur un paper :
https://hal.telecom-paris.fr/hal-02367908/document
## Il a été réalisé en collaboration avec Frederic Haykal 
donc une partie du code sera commune à nos deux rendus.

# Idée maitresse de la solution :
Nous allons produire un modèle par base.\
Ce modèle va apprendre à produire la distance entre la base et les messages reçus par cette base en s'appuyant sur le rssi.\
Le modèle va modéliser les perturbations terrain afin de pouvoir prédire une distance avec un rssi non fiable.

### Explication de l'idée
Cas du GPS : le rssi donne la puissance recu du signal. ainsi avec cet valeur on peut définir la distance parcouru par le signal.\
Notre cas : Mais par construction dans notre cas ce rssi est perturbé par autre chose que la distance (le terrain). Nous utiliserons donc du Machine Learning pour apprendre ces perturbations et arriver a calculer cette distance.

### de plus
On remarque que si l'on met les distances au lieu des 1 dans la df_feature on a de très bon résultat.\
Notre objectif sera donc de créer cette matrice dans le jeu de test en s'appuyant sur un modèle appris sur le train.

Un dernier model prédira la position du message en s'appuyant sur cette nouvelle matrice.\
NB : nous rajouterons également les barycentres venant des autres jupyter pour encore augmenter la précision.

Comme on peut le voir à la fin, malgré un overfit non négligeable les résultats sont prometteurs

In [12]:
import pandas as pd
import numpy as np
from IotTools import *
#from IpyTools import *
pd.options.mode.chained_assignment = None  # default='warn'
import pickle
from vincenty import vincenty

from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.tree import ExtraTreeRegressor
from sklearn.ensemble import VotingRegressor

from sklearn.model_selection import cross_val_predict

In [13]:
df_mess_train = pd.read_csv('mess_train_list.csv')
df_mess_test = pd.read_csv('mess_test_list.csv')
pos_train = pd.read_csv('pos_train_list.csv')
listOfBs = np.union1d(df_mess_train.bsid.unique(),df_mess_test.bsid.unique())

In [14]:
size = len(listOfBs)
size

259

# On corrige les bases outliers

In [15]:
X_train= Correct_Bases (df_mess_train)
X_train[X_train.bs_lat>64].shape[0];

Nous avons 27 bases outliers
Base 9949 non vu
il reste 0 base avec lat >60


In [16]:
X_test= Correct_Bases (df_mess_test)
X_test[X_test.bs_lat>64].shape[0];

Nous avons 23 bases outliers
Correction manuelle de la bsid 9949
il reste 0 base avec lat >60


# On créer la liste des coordonnées de toutes les bases

In [17]:
#On crééer une matrice contenant les coordonnées des bases
base_coord = pd.concat([X_train,X_test]).drop_duplicates(subset='bsid')[['bsid','bs_lat','bs_lng']].set_index('bsid')

# On créer une feature distance

In [18]:
msg_coord = (pos_train.set_index(keys=df_mess_train.messid.values).reset_index()).groupby('index').mean()
X_train['dist']=0
for i in X_train.index:
    bs_Coord = (X_train.iloc[i].bs_lat,X_train.iloc[i].bs_lng)
    msg_Coord = (msg_coord.loc[X_train.iloc[i].messid].lat,msg_coord.loc[X_train.iloc[i].messid].lng)
    X_train.loc[i,'dist']=vincenty(bs_Coord,msg_Coord)
X_train.head(1)

,messid,bsid,did,nseq,rssi,time_ux,bs_lat,bs_lng,dist
0,573bf1d9864fce1a9af8c5c9,2841,473335.0,0.5,-121.5,1.463546e+12,39.617794,-104.954917,1.270478


# La fct feat matrice propre a notre cas où l'on fait apparaitre le Rssi

In [19]:
def feat_mat_const(df, listOfBs):
    """On a rajouté une colonne contenant le nombre de détection par msg
    cela servira à identifié la pondération de la prédiction
    """
    df_group = df.groupby("messid", as_index=False)
    nb_mess = len(df.messid.unique())
    df_feat = pd.DataFrame(np.zeros((nb_mess, len(listOfBs))), columns=listOfBs, index=df.messid.unique())
    id_list = [0] * nb_mess

    for i, (key, elmt) in enumerate(df_group):
        df_feat.loc[key,elmt.bsid]= elmt.rssi.values #ici on met le rssi au lieu des 1 
        id_list[i] = key
        
    """Rajout des BaryCentres"""
            
    a =df.rssi.values
    df['rssi_reshape'] = (10**(a/10))
    df['bs_lat_pond'] = df.bs_lat * df.rssi_reshape
    df['bs_lng_pond'] = df.bs_lng * df.rssi_reshape
    BCW_lat = df.groupby('messid').bs_lat_pond.sum()/df.groupby('messid').rssi_reshape.sum()
    BCW_lat.name='BCW_lat'
    BCW_lng = df.groupby('messid').bs_lng_pond.sum()/df.groupby('messid').rssi_reshape.sum()
    BCW_lng.name='BCW_lng'

    #Pour le barycentre ajout min , max + mean 

    BC_lat = df.groupby('messid').agg(['mean', 'min', 'max'])[['bs_lat']]
    BC_lat.columns = BC_lat.columns.droplevel()

    BC_lng = df.groupby('messid').agg(['mean', 'min', 'max'])[['bs_lng']]
    BC_lng.columns = BC_lng.columns.droplevel()

    dev_lat =df.groupby(['did','messid']).agg(['mean', 'min', 'max'])[['bs_lat']].reset_index()
    dev_lat.columns = dev_lat.columns.droplevel()
    dev_lat.columns= ['did','messid','bs_l_did_mean', 'bs_l_did_min', 'bs_l_did_max']
    dev_lat=dev_lat.drop('did',axis=1)

    dev_lng =df.groupby(['did','messid']).agg(['mean', 'min', 'max'])[['bs_lng']].reset_index()
    dev_lng.columns = dev_lng.columns.droplevel()
    dev_lng.columns= ['did','messid','bs_L_did_mean', 'bs_L_did_min', 'bs_L_did_max']
    dev_lng=dev_lng.drop('did',axis=1)

    #On fait apparaitre une colonne contenant le messid pour la jointure

    df_feat = df_feat.reset_index()
    df_feat.rename(columns={'index':'messid'}, inplace=True)

    df_feat=df_feat.join(BCW_lat, on ='messid',how='left')
    df_feat=df_feat.join(BCW_lng, on ='messid',how='left')

    df_feat= pd.merge(df_feat,BC_lat,on='messid')
    df_feat= pd.merge(df_feat,BC_lng,on='messid')

    df_feat= pd.merge(df_feat,dev_lat,on='messid')
    df_feat= pd.merge(df_feat,dev_lng,on='messid')

    #On remet le messid dans l'index
    df_feat.set_index('messid',inplace=True)
    
    return df_feat, id_list

In [20]:
df_feat, id_list=feat_mat_const(X_train, listOfBs)

y_full = ground_truth_const(X_train, pos_train, id_list)
y_full.shape,df_feat.shape

((6068, 3), (6068, 273))

%%time
"""Cette celluel tourne pendant 2h !!!!!"""
"""Le résultat est stocké en pickle voir plus bas"""


#Code produit ensemble donc tu peux copier bêtement
Models={}
df_temp = df_feat[df_feat.columns[:-14]] #Ici on récupère la matrice df_feat sans columns supplémentaire
#Pour chaque base
for idx,base in enumerate(df_temp.columns):
    print(f"Base : {base}")
    
    #Prod de la sous matrice df_feat (lié à la base)
    df_feat_reduce=df_temp[df_temp[base]!=0]
    print(f"Nb message : {df_feat_reduce.shape[0]}")
    
    #On ne produit pas de modèle pour les bases non vu dans le train
    if df_feat_reduce.shape[0] :
        y=[];X=[]
        #On parcours tous les messages identifiés à une base
        for i,k in enumerate(df_feat_reduce.index):
            for j in df_temp.columns: #Je parcours toutes les bases et construit la matrice 259x259
                #print("tip")
                X.append(abs(df_temp.loc[k,j]-df_temp.loc[k,df_temp.columns].values))
                y.append(vincenty(base_coord.loc[j],msg_coord.loc[k]))
                #print("top")
        y=np.array(y)
        X=np.array(X)
        #On produit un modèle pour la base en question on note également l'indice de la base 
        model = xgb.XGBRegressor().fit(X,y);
        Models[base] = [idx,model]
        
        """ce bout de code ne sert QUE a vérifier si la méthode est bonne mais n'est pas nécessaire"""

        y_pred= Models[base][1].predict(X)
        #Calcul de le MAE
        sol=[]
        for j in range(y_pred.shape[0]//size):
            sol.append(y_pred[j*size:(j+1)*size][Models[base][0]])
        a =np.array(sol)
        b = X_train[X_train.bsid == base].dist.values 
        
        print(f"MAE de la base : {base} : {(np.abs(a-b)).mean()}")

    else :
        print(f"Base {base} en échec")

"""NE PAS RUNNER"""
with open (f"Models.pkl","wb") as fp :
    pickle.dump(Models, fp)

In [21]:
""" Runner cette cellule pour charger les Models"""
with open (f"Models.pkl","rb") as fp :
    Models = pickle.load(fp)

# Prédiction sur le Train pour qualifier la solution et le fit

In [22]:
df=X_train
df_feat, id_list=feat_mat_const(df, listOfBs)
df_temp = df_feat[df_feat.columns[:-14]]
df_group = df.groupby("messid", as_index=False)
nb_mess = len(df.messid.unique())
df_feat_new = pd.DataFrame(np.zeros((nb_mess, len(listOfBs))), columns=listOfBs, index=df.messid.unique())

%%time
for idx,base in enumerate(df_temp.columns):
    print(f"Base : {base}")
    #Prod de la sous matrice df_feat (lié à la base)
    df_feat_reduce=df_temp[df_temp[base]!=0]
    print(f"Nb message : {df_feat_reduce.shape[0]}")
    if df_feat_reduce.shape[0]:
        if Models.get(base,False) :
            X=[]
            #On parcours tous les messages identifié à une base
            for i,k in enumerate(df_feat_reduce.index):
                for j in df_temp.columns: #Je parcours toutes les bases et construit la matrice 259x259
                    X.append(abs(df_temp.loc[k,j]-df_temp.loc[k,df_temp.columns]).values)
            X=np.array(X)
            y= Models[base][1].predict(X)
            sol=[]
            for j in range(y.shape[0]//size):
                sol.append(y[j*size:(j+1)*size][Models[base][0]])
            df_feat_new.loc[df_feat_reduce.index.values,base]=np.array(sol)
        else:
            print(f"Base non modélisée {base}")
            """Ici on pourrait prévoire un amélioration qui une fois le message prédit on construit le modèle"""
    else :
        print(f"Base sans message {base}")

#Je rajoute le calcul des barycente que l'on trouve dans le df_feat
df_feat_new_TRAIN = pd.merge(df_feat_new,df_feat[df_feat.columns[-14:]],right_index=True,left_index=True) 

"""NE PAS RUNNER"""
with open (f"df_feat_new_TRAIN.pkl","wb") as fp :
    pickle.dump(df_feat_new_TRAIN, fp)

In [24]:
""" Runner cette cellule pour charger la df Feat du X_train"""
with open (f"df_feat_new_TRAIN.pkl","rb") as fp :
    df_feat_new_TRAIN = pickle.load(fp)

# Qualification avec les hypers paramètres trouvés

In [25]:
df_params = pd.read_csv("best_params.csv")

In [26]:
r1 = RandomForestRegressor(**get_hyperparameter('RandomForestRegressor', 'lng'))
r2 = GradientBoostingRegressor(**get_hyperparameter('GradientBoostingRegressor', 'lng'))
r3 = ExtraTreeRegressor(**get_hyperparameter('ExtraTreeRegressor', 'lng'))
r4 = xgb.XGBRegressor(**get_hyperparameter('XGBRegressor', 'lng'))
r5 = BaggingRegressor(**get_hyperparameter('BaggingRegressor', 'lng'))
Vr_lng = VotingRegressor(estimators=[('Et',r1),('Rf',r2),('Gb',r3),('Xg',r4),('Xdg',r5)])


r1 = RandomForestRegressor(**get_hyperparameter('RandomForestRegressor', 'lat'))
r2 = GradientBoostingRegressor(**get_hyperparameter('GradientBoostingRegressor', 'lat'))
r3 = ExtraTreeRegressor(**get_hyperparameter('ExtraTreeRegressor', 'lat'))
r4 = xgb.XGBRegressor(**get_hyperparameter('XGBRegressor', 'lat'))
r5 = BaggingRegressor(**get_hyperparameter('BaggingRegressor', 'lat'))
Vr_lat = VotingRegressor(estimators=[('Et',r1),('Rf',r2),('Gb',r3),('Xg',r4),('Xdg',r5)])

y_pred_lng = cross_val_predict(Vr_lng, df_feat_new_TRAIN, y_full.lng, cv=3)
y_pred_lat = cross_val_predict(Vr_lat, df_feat_new_TRAIN, y_full.lat, cv=3)

err_vec = Eval_geoloc(y_full.lat , y_full.lng, y_pred_lat, y_pred_lng)
np.percentile(err_vec, 80)

2.8549834

# Prédiction du Test

In [27]:
df=X_test
df_feat, id_list=feat_mat_const(df, listOfBs)
df_temp = df_feat[df_feat.columns[:-14]]
df_group = df.groupby("messid", as_index=False)
nb_mess = len(df.messid.unique())
df_feat_new = pd.DataFrame(np.zeros((nb_mess, len(listOfBs))), columns=listOfBs, index=df.messid.unique())

%%time
for idx,base in enumerate(df_temp.columns):
    print(f"Base : {base}")
    #Prod de la sous matrice df_feat (lié à la base)
    df_feat_reduce=df_temp[df_temp[base]!=0]
    print(f"Nb message : {df_feat_reduce.shape[0]}")
    if df_feat_reduce.shape[0]:
        if Models.get(base,False) :
            X=[]
            #On parcours tous les messages identifié à une base
            for i,k in enumerate(df_feat_reduce.index):
                for j in df_temp.columns: #Je parcours toutes les bases et construit la matrice 259x259
                    X.append(abs(df_temp.loc[k,j]-df_temp.loc[k,df_temp.columns]).values)
            X=np.array(X)
            y= Models[base][1].predict(X)
            sol=[]
            for j in range(y.shape[0]//size):
                sol.append(y[j*size:(j+1)*size][Models[base][0]])
            df_feat_new.loc[df_feat_reduce.index.values,base]=np.array(sol)
        else:
            print(f"Base non modélisée {base}")
            """Ici on pourrait prévoire un amélioration qui une fois le message prédit on construit le modèle"""
    else :
        print(f"Base sans message {base}")

df_feat_new_TEST = pd.merge(df_feat_new,df_feat[df_feat.columns[-14:]],right_index=True,left_index=True) 

"""NE PAS RUNNER"""
with open (f"df_feat_new_TEST.pkl","wb") as fp :
    pickle.dump(df_feat_new_TEST, fp)

In [28]:
""" Runner cette cellule pour charger la df Feat du X_test"""
with open (f"df_feat_new_TEST.pkl","rb") as fp :
    df_feat_new_TEST = pickle.load(fp)

In [29]:
r1 = RandomForestRegressor(**get_hyperparameter('RandomForestRegressor', 'lng'))
r2 = GradientBoostingRegressor(**get_hyperparameter('GradientBoostingRegressor', 'lng'))
r3 = ExtraTreeRegressor(**get_hyperparameter('ExtraTreeRegressor', 'lng'))
r4 = xgb.XGBRegressor(**get_hyperparameter('XGBRegressor', 'lng'))
r5 = BaggingRegressor(**get_hyperparameter('BaggingRegressor', 'lng'))
Vr_lng = VotingRegressor(estimators=[('Et',r1),('Rf',r2),('Gb',r3),('Xg',r4),('Xdg',r5)])


r1 = RandomForestRegressor(**get_hyperparameter('RandomForestRegressor', 'lat'))
r2 = GradientBoostingRegressor(**get_hyperparameter('GradientBoostingRegressor', 'lat'))
r3 = ExtraTreeRegressor(**get_hyperparameter('ExtraTreeRegressor', 'lat'))
r4 = xgb.XGBRegressor(**get_hyperparameter('XGBRegressor', 'lat'))
r5 = BaggingRegressor(**get_hyperparameter('BaggingRegressor', 'lat'))
Vr_lat = VotingRegressor(estimators=[('Et',r1),('Rf',r2),('Gb',r3),('Xg',r4),('Xdg',r5)])

Vr_lng.fit(df_feat_new_TRAIN, y_full.lng)
Vr_lat.fit(df_feat_new_TRAIN, y_full.lat)

y_pred_lng = Vr_lng.predict(df_feat_new_TEST)
y_pred_lat = Vr_lat.predict(df_feat_new_TEST)

#Il faut produire le fichier solution

In [30]:
y_pred_lng

array([-105.11921928, -105.08057484, -105.0147477 , ..., -105.03055694,
       -105.02294792, -105.02297024])

In [31]:
y_pred_lat

array([39.72010534, 39.77769262, 39.68975325, ..., 39.68913253,
       39.67764977, 39.68792618])

In [32]:
sol = pd.DataFrame({'lat':y_pred_lat,'lng':y_pred_lng,'messid':df_feat_new_TEST.index})

In [34]:
sol.to_csv("calligaro.csv",index=False)

In [35]:
test=pd.read_csv("pred_pos_test_list.csv")

In [37]:
test.lat

0       39.772912
1       39.774800
2       39.678750
3       39.773684
4       39.678750
          ...    
5289    39.745662
5290    39.778171
5291    39.692797
5292    39.741485
5293    39.699054
Name: lat, Length: 5294, dtype: float64

In [38]:
err_vec = Eval_geoloc(test.lat , test.lng, y_pred_lat, y_pred_lng)
np.percentile(err_vec, 80)

6.072840800000007

In [28]:
from ipyleaflet import Map, basemaps
msg_test = pd.DataFrame({'lat':y_pred_lat,'lng':y_pred_lng},index=df_feat_new_TEST.index)

barycentre = ((msg_test.lat.max()+msg_test.lat.min())/2,(msg_test.lng.min()+msg_test.lng.max())/2)

m = Map(center=barycentre, zoom=3, basemap = basemaps.OpenStreetMap.Mapnik)

message = Give_Marker_Cluster(msg_test)

m.add_layer(message)

m

OSError: [WinError 126] Le module spécifié est introuvable